# 必要な計算環境

* Linux OS
* Pascal, Volta, Turing, 或いは NVIDIA Ampere 世代 GPU
* Nvidia Driver
* Docker

# 事前準備

* [NGC](https://catalog.ngc.nvidia.com/?filters=&orderBy=weightPopularDESC&query=&page=&pageSize=)の登録と[APIキー](https://org.ngc.nvidia.com/setup/api-key)の取得は済みましたか？
* [Weights and Biases]((https://docs.wandb.ai/quickstart)) の登録と[APIキー](https://docs.wandb.ai/guides/track/public-api-guide#authentication)の取得は済みましたか？

### NGC のセットアップ
NGCからモデルをダウンロードしたい場合は、APIキーが必要になりますので、取得してください。
### Weights and Biases のセットアップ
モデルのトレーニング進行状況やチャートは、 [Weights and Biases](https://docs.wandb.ai/quickstart) を通じて可視化できます。ログを有効にするために、 [API キー](https://docs.wandb.ai/guides/track/public-api-guide#authentication)を設定してください。

# GPUの確認

In [1]:
!nvidia-smi

Wed May 15 04:08:12 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.129.03             Driver Version: 535.129.03   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA A100-SXM4-80GB          On  | 00000000:87:00.0 Off |                    0 |
| N/A   30C    P0              56W / 400W |      7MiB / 81920MiB |      0%      Default |
|                                         |                      |             Disabled |
+-----------------------------------------+----------------------+--

# データの前処理

DNABERT のトレーニングは オリジナルDNABERTと同じ参照バージョンのヒトゲノム、NIHからダウンロードされたGRCh38.p13を使用しています。クロモソームはまず連続したセクション（例えば、Nで切断された部分）に分割され、訓練でサンプリングされる「空の」シーケンスが除去されます。その後、ゲノムのスライスがランタイムでサンプリングされ、訓練のためにモデルに提供されます。デフォルト設定では、chr1からchr19までが訓練に使用され、chr20とchr21は保留データとして取り置かれます。また、chr22もさらなる評価のために保留されています。

次に、下記の場所にあるコードを実行してデータの前処理を行いますが、その前に、データのパスを設定する必要があります。

/workspace/bionemo/examples/dna/dnabert/pretrain.py

データのパスを正しく設定するために、下記のYAML設定ファイルを変更するか、データの前処理やモデルトレーニングを行う際にコマンドの一部としてそれらのパラメータを更新することができます。

/workspace/bionemo/examples/dna/dnabert/conf/dnabert_base_config.yaml

では、以下のコマンドでデータの前処理を行います。

In [1]:
cd /workspace/bionemo 

/workspace/bionemo


In [2]:
!python examples/dna/dnabert/pretrain.py\
 --config-path=conf\
 --config-name=dnabert_xsmall\
 ++do_training=False\
 ++model.data.dataset_path=/workspace/bionemo/examples/dna/dnabert/data

[NeMo I 2024-05-16 01:26:19 megatron_hiddens:110] Registered hidden transform sampled_var_cond_gaussian at bionemo.model.core.hiddens_support.SampledVarGaussianHiddenTransform
[NeMo I 2024-05-16 01:26:19 megatron_hiddens:110] Registered hidden transform interp_var_cond_gaussian at bionemo.model.core.hiddens_support.InterpVarGaussianHiddenTransform
[NeMo W 2024-05-16 01:26:19 nemo_logging:349] /usr/local/lib/python3.10/dist-packages/hydra/_internal/defaults_list.py:251: UserWarning: In 'dnabert_xsmall': Defaults list is missing `_self_`. See https://hydra.cc/docs/1.2/upgrades/1.0_to_1.1/default_composition_order for more information
      warnings.warn(msg, UserWarning)
    
[NeMo W 2024-05-16 01:26:19 nemo_logging:349] /usr/local/lib/python3.10/dist-packages/hydra/_internal/hydra.py:119: UserWarning: Future Hydra versions will no longer change working directory at job runtime by default.
    See https://hydra.cc/docs/1.2/upgrades/1.1_to_1.2/changes_to_job_working_dir/ for more informat

-- で始まるパラメータは、コマンドライン引数として pretrain.py に渡されます。例えば、 config-path と config-name は、設定 YAML ファイルのフォルダとyaml ファイル名を指定します。このパスは pretrain.py に対して相対的です。conf はexamples/dna/dnabert/confを指し、dnabert_xsmall はexamples/dna/dnabert/conf/dnabert_xsmall.yaml を指します。 


++ で始まるパラメータは、YAML ファイルで設定可能です。 例えば、dnabert_base_config.yaml から継承された dnabert_xsmall.yaml では、以下のパラメータを見つけることができます： 

* do_training: データの前処理のみを行い、トレーニングは行わないように False に設定します。 
* model.data.dataset_path: 前処理された GRCh38.p13 データの出力ディレクトリへのパスを指定します。このフォルダはトレイン、検証、テストの分割を含むことになります。 

また、上記のようにコマンドラインを通じて引数を上書きするのではなく、直接 YAML ファイルを変更することもできます。処理が完了すると、前処理されたデータは /workspace/bionemo/examples/dna/dnabert/data にあります。

もし独自のデータを利用して事前学習、ファインチューニングや推論をしたい場合は、パスを/workspace/bionemo/mydata/ に指定してください。ただし、データの構造やフォーマットをサンプルデータに揃える必要があります。

# モデルの事前学習

GRCh38.p13 データのダウンロードと前処理が完了したので、DNABERT モデルの事前学習を開始することができます。

では、以下のコマンドをターミナルに入力して、モデルを事前学習します。

In [3]:
cd /workspace/bionemo 

/workspace/bionemo


In [4]:
!python examples/dna/dnabert/pretrain.py\
 --config-path=conf\
 --config-name=dnabert_xsmall\
 ++do_training=True\
 ++model.data.dataset_path=/workspace/bionemo/examples/dna/dnabert/data\
 ++model.micro_batch_size=64\
 ++trainer.max_steps=100

[NeMo I 2024-05-16 01:30:33 megatron_hiddens:110] Registered hidden transform sampled_var_cond_gaussian at bionemo.model.core.hiddens_support.SampledVarGaussianHiddenTransform
[NeMo I 2024-05-16 01:30:33 megatron_hiddens:110] Registered hidden transform interp_var_cond_gaussian at bionemo.model.core.hiddens_support.InterpVarGaussianHiddenTransform
[NeMo W 2024-05-16 01:30:33 nemo_logging:349] /usr/local/lib/python3.10/dist-packages/hydra/_internal/defaults_list.py:251: UserWarning: In 'dnabert_xsmall': Defaults list is missing `_self_`. See https://hydra.cc/docs/1.2/upgrades/1.0_to_1.1/default_composition_order for more information
      warnings.warn(msg, UserWarning)
    
[NeMo W 2024-05-16 01:30:34 nemo_logging:349] /usr/local/lib/python3.10/dist-packages/hydra/_internal/hydra.py:119: UserWarning: Future Hydra versions will no longer change working directory at job runtime by default.
    See https://hydra.cc/docs/1.2/upgrades/1.1_to_1.2/changes_to_job_working_dir/ for more informat

パラメータの説明：
* do_training: モデルをトレーニングするために True に設定します。これはデータが前処理されたことを前提としています。
* model.data.dataset_path: トレーニング/検証/テスト分割が含まれる前処理済みの GRCh38.p13 データフォルダのパスを指定します。
* trainer.devices: 使用するGPUの数を指定します。
* model.micro_batch_size: バッチサイズを設定します。メモリエラーが発生しない限り、これをできるだけ増やします。
* trainer.max_steps: トレーニングの最大ステップ数を指定します。デモのために 100 に設定しました。1 ステップ = 1 バッチの処理です。まず、total_batches = サンプル総数 / バッチサイズを計算します。N エポックでトレーニングしたい場合は、max_steps をN * total_batches に設定します。

トレーニングされた結果が /workspace/bionemo/results/nemo_experiments/ に保存されます。


# ファインチューニング

BioNemo Framework のサンプルコードはスプライスサイト予測という下流タスクを提供しています。次に、Ensemble の GRCh38.p13 バージョン99のアノテーションを使用して、スプライスサイト予測タスクでモデルをファインチューニングします。10,000のドナーサイト、10,000のアクセプターサイト、そして遺伝子体からの10,000のランダム負サイトがサンプリングされ、訓練用（80%）、検証用（10%）、およびテスト用（10%）に分割されます。

下記のコマンドで下流タスクのデータをダウンロードします。

In [ ]:
cd /workspace/bionemo

In [ ]:
!python examples/dna/dnabert/downstream_splice_site.py\
 --config-path=conf\
 --config-name=dnabert_config_splice_site\
 ++do_training=False\
 ++model.data.dataset_path=/workspace/bionemo/examples/dna/dnabert/data

BioNeMo Framework では、事前学習されたモデルのチェックポイントが提供されています。例えば、ESM-1nv、ESM-2nv（650m、3b、15b）、ProtT5nv、MegaMolBART などです。これらのモデルの重みは NVIDIA の NGC からダウンロードできます。

次に、事前学習済みモデルをダウンロードします。モデルをダウンロードするため、ngc をインストールして、ngc config を設定する必要があります。
01_protein_LLM ノートブックで ngc confit の設定が完了した場合は、このステップをスキップして大丈夫です。

In [12]:
!wget -q -O /tmp/ngccli_linux.zip --content-disposition https://api.ngc.nvidia.com/v2/resources/nvidia/ngc-apps/ngc_cli/versions/3.38.0/files/ngccli_linux.zip && unzip -o /tmp/ngccli_linux.zip -d /tmp && chmod u+x /tmp/ngc-cli/ngc && rm /tmp/ngccli_linux.zip

Archive:  /tmp/ngccli_linux.zip
  inflating: /tmp/ngc-cli/boto3/examples/s3.rst  
  inflating: /tmp/ngc-cli/boto3/examples/cloudfront.rst  
  inflating: /tmp/ngc-cli/boto3/data/dynamodb/2012-08-10/resources-1.json  
  inflating: /tmp/ngc-cli/boto3/data/sqs/2012-11-05/resources-1.json  
  inflating: /tmp/ngc-cli/boto3/data/opsworks/2013-02-18/resources-1.json  
  inflating: /tmp/ngc-cli/boto3/data/iam/2010-05-08/resources-1.json  
  inflating: /tmp/ngc-cli/boto3/data/ec2/2014-10-01/resources-1.json  
  inflating: /tmp/ngc-cli/boto3/data/ec2/2015-10-01/resources-1.json  
  inflating: /tmp/ngc-cli/boto3/data/ec2/2015-03-01/resources-1.json  
  inflating: /tmp/ngc-cli/boto3/data/ec2/2016-09-15/resources-1.json  
  inflating: /tmp/ngc-cli/boto3/data/ec2/2016-04-01/resources-1.json  
  inflating: /tmp/ngc-cli/boto3/data/ec2/2015-04-15/resources-1.json  
  inflating: /tmp/ngc-cli/boto3/data/ec2/2016-11-15/resources-1.json  
  inflating: /tmp/ngc-cli/boto3/data/s3/2006-03-01/resources-1.json  

そして、ターミナルを開いて、下記のコマンドを入力して、 ngc config set をしてください。

/tmp/ngc-cli/ngc config set

<順番にAPI キー、CLI output format, org, team, aceを入力してください。API キーはNGCのAPIキーを入力してください。Org は ’no-org’ 以外のものを選択して、その他は「Enter」でデフォルト値を入力すれば大丈夫です>

最後に、下記のコマンドを入力すれば、モデルをダウンロードできます。

In [13]:
!python download_models.py --download_dir /workspace/bionemo/models dnabert

Running command: ngc --version

/bin/dash: 1: ngc: not found

Installing NGC CLI to /tmp
Running command: wget -q -O /tmp/ngccli_linux.zip --content-disposition https://api.ngc.nvidia.com/v2/resources/nvidia/ngc-apps/ngc_cli/versions/3.38.0/files/ngccli_linux.zip && unzip -o /tmp/ngccli_linux.zip -d /tmp && chmod u+x /tmp/ngc-cli/ngc && rm /tmp/ngccli_linux.zip

Archive:  /tmp/ngccli_linux.zip
  inflating: /tmp/ngc-cli/boto3/examples/s3.rst  
  inflating: /tmp/ngc-cli/boto3/examples/cloudfront.rst  
  inflating: /tmp/ngc-cli/boto3/data/dynamodb/2012-08-10/resources-1.json  
  inflating: /tmp/ngc-cli/boto3/data/sqs/2012-11-05/resources-1.json  
  inflating: /tmp/ngc-cli/boto3/data/opsworks/2013-02-18/resources-1.json  
  inflating: /tmp/ngc-cli/boto3/data/iam/2010-05-08/resources-1.json  
  inflating: /tmp/ngc-cli/boto3/data/ec2/2014-10-01/resources-1.json  
  inflating: /tmp/ngc-cli/boto3/data/ec2/2015-10-01/resources-1.json  
  inflating: /tmp/ngc-cli/boto3/data/ec2/2015-03-01/resourc

ダウンロードされたモデルは .nemo というファイルで、 /workspace/bionemo/models に保存されます。
そして、以下の YAML ファイルを開いて、必要に応じてパラメータを設定してください。

/workspace/bionemo/examples/dna/dnabert/conf/downstream_splice_site.py.yaml

そして、ターミナルで下記のコマンドでファインチューニングを実行します。

In [5]:
cd /workspace/bionemo

/workspace/bionemo


In [6]:
!python examples/dna/dnabert/downstream_splice_site.py\
 --config-path=conf\
 --config-name=dnabert_config_splice_site\
 ++model.data.dataset_path=/workspace/bionemo/examples/dna/dnabert/data\
 ++model.data.train_file=/workspace/bionemo/examples/dna/dnabert/data/train.csv\
 ++model.data.val_file=/workspace/bionemo/examples/dna/dnabert/data/val.csv\
 ++model.data.predict_file=/workspace/bionemo/examples/dna/dnabert/data/test.csv\
 ++model.data.fasta_directory=/workspace/bionemo/examples/dna/dnabert/data/

[NeMo I 2024-05-16 01:36:53 megatron_hiddens:110] Registered hidden transform sampled_var_cond_gaussian at bionemo.model.core.hiddens_support.SampledVarGaussianHiddenTransform
[NeMo I 2024-05-16 01:36:53 megatron_hiddens:110] Registered hidden transform interp_var_cond_gaussian at bionemo.model.core.hiddens_support.InterpVarGaussianHiddenTransform
[NeMo W 2024-05-16 01:36:53 nemo_logging:349] /usr/local/lib/python3.10/dist-packages/hydra/_internal/defaults_list.py:251: UserWarning: In 'dnabert_config_splice_site': Defaults list is missing `_self_`. See https://hydra.cc/docs/1.2/upgrades/1.0_to_1.1/default_composition_order for more information
      warnings.warn(msg, UserWarning)
    
[NeMo W 2024-05-16 01:36:53 nemo_logging:349] /usr/local/lib/python3.10/dist-packages/hydra/_internal/hydra.py:119: UserWarning: Future Hydra versions will no longer change working directory at job runtime by default.
    See https://hydra.cc/docs/1.2/upgrades/1.1_to_1.2/changes_to_job_working_dir/ for m

# 推論

下記の YAML ファイルを開いて、

/workspace/bionemo/examples/dna/dnabert/conf/infer.yaml

以下の情報を更新してください。提供する事前学習済みモデルを使用するか、自分でトレーニングしたモデルを使用することができます。

In [ ]:
downstream_task:
 restore_from_path: "${oc.env:BIONEMO_HOME}/models/dna/dnabert/dnabert_86M.nemo" # 事前学習済みモデルのパス

そして、下記のコマンドで推論を行います。

In [ ]:
import warnings

warnings.filterwarnings('ignore')
warnings.simplefilter('ignore')

In [ ]:
from pathlib import Path
import os

try:
    BIONEMO_HOME: Path = Path(os.environ['BIONEMO_HOME']).absolute()
except KeyError:
    print("Must have BIONEMO_HOME set in the environment! See docs for instructions.")
    raise

config_path = BIONEMO_HOME / "examples" / "dna" / "dnabert" / "conf"
print(f"Using model configuration at: {config_path}")
assert config_path.is_dir()

In [ ]:
seqs = [
    'CACATGCTAGCGCGTCGGGGTGGAGGCGTGGCGCAGGCGCAGAGAGGCGCGCCGCGCCG', 
    'GCGCAGGCGCAGAGACACATGCTACCGCGTCCAGGGGTGGAGGCGTGGCGCAGGCGCAG',
    'GCAAAGTCGCACGGCGCCGGGCTGGGGCGGGGGGAGGGTGGCGCCGTGCACGCGCAGAA',
    'CGCAGAGACGGGTAGAACCTCAGTAATCCGAAAAGCCGGGATCGACCGCCCCTTGCTTG',
]

In [ ]:
from bionemo.utils.hydra import load_model_config

cfg = load_model_config(config_name="infer.yaml", config_path=config_path)

In [ ]:
from bionemo.triton.utils import load_model_for_inference
from bionemo.model.dna.dnabert.infer import DNABERTInference

inferer = load_model_for_inference(cfg, interactive=True)

print(f"Loaded a {type(inferer)}")
assert isinstance(inferer, DNABERTInference)

In [ ]:
hidden_states, pad_masks = inferer.seq_to_hiddens(seqs)
print(f"{hidden_states.shape=}")
print(f"{pad_masks.shape=}")
assert tuple(hidden_states.shape) == (2, 43, 1280)
assert tuple(pad_masks.shape) == (2, 43)

In [ ]:
embeddings = inferer.hiddens_to_embedding(hidden_states, pad_masks)
print(f"{embeddings.shape=}")
assert tuple(embeddings.shape) == (2, 1280)

In [ ]:
embeddings = inferer.seq_to_embeddings(seqs)
print(f"{embeddings.shape=}")
assert tuple(embeddings.shape) == (2, 1280)